# Interactive loading of PHOENIX NewEra JWST synthetic stellar spectra h5 files
Model grids published in The NewEra model grid ([Hauschildt _et al_., 2025](https://ui.adsabs.harvard.edu/abs/2025arXiv250417597H))

Data and sample code published at [DOI: 10.25592/uhhfdm.16727](https://doi.org/10.25592/uhhfdm.17156).

This loosely derived from `example_read_HSR_H5.py`.

In [1]:
from pathlib import Path
import requests

import astropy.units as u

# Specific requirement for these files
import h5py
import f90nml

In [2]:
def make_NewEra_filename(teff, logg, zscale=0, alpha_scale=0) -> Path:
    """
    Generate the (LTE) NewEra HSR filename for given criteria.
    """
    if zscale == +0: # Ensure zero Z is prefixed with -
        zscale = -0.0
        
    if alpha_scale == 0.0:
        job_name = f'lte{teff:0=5.0f}{-logg:3.2f}{zscale:0=+4.1f}'
    else:
        job_name = f'lte{teff:0=5.0f}{-logg:3.2f}{zscale:0=+4.1f}.alpha={alpha_scale:0=+3.1f}'
    return f'{job_name}.PHOENIX-NewEra-ACES-COND-2023.HSR.h5'

def download_NewEra_file(filename: str, save_dir: Path):
    """
    Will download the requested file from the NewEra archive and save to the requested
    save directory, having first checked that the target does not already exist.
    """
    if isinstance(save_dir, str):
        save_dir = Path(save_dir)

    if not save_dir.exists():
        raise ValueError("save_dir does not exist")
    if not save_dir.is_dir():
        raise ValueError("save_dir is not a directory")
    
    if not (save_dir / filename).exists():
        try:
            url = f"https://www.fdr.uni-hamburg.de/record/16738/files/{filename}?download=1"
            print(f"Downloading {filename} from {url}")
            with requests.get(url, stream=True) as response:
                response.raise_for_status()  # Check for request errors

                # Open a local file with write-binary mode
                with open(save_dir / filename, 'wb') as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        file.write(chunk)
                print(f'Saved to {save_dir / filename}')
        except requests.exceptions.RequestException as e:
            print('Error for:', e)
    else:
        print(f"File {save_dir / filename} exists.")

### Parse the SPECTRA h5 file
The data files are in the [PHOENIX-NewEra-JWST-SPECTRA.tar.gz](https://www.fdr.uni-hamburg.de/record/17156/files/PHOENIX-NewEra-JWST-SPECTRA.tar.gz?download=1)
archive published at [DOI: 10.25592/uhhfdm.16727](https://doi.org/10.25592/uhhfdm.17156).

> **Note**: the archive is ~11.8 GB is size

Download on demand the corresponding data file.

In [3]:
# Selection criteria
Teff = 12000
logg = 3.5
Z = 0.0

In [4]:
save_dir = Path("./libs/data/newera/")
filename = make_NewEra_filename(Teff, logg, Z)
download_NewEra_file(filename, save_dir)

File libs/data/newera/lte12000-3.50-0.0.PHOENIX-NewEra-ACES-COND-2023.HSR.h5 exists.


In [5]:
with h5py.File(save_dir / filename, "r") as fh5:
    # read the HSR spectrum: 
    wl = fh5['/PHOENIX_SPECTRUM/wl'][()]                # wavelength in Angstroem, vacuum
    flux = 10.**fh5['/PHOENIX_SPECTRUM/flux'][()]       # flux, stored as log10 values
    bb_flux = 10.**fh5['/PHOENIX_SPECTRUM/bb'][()]      # BB flux, stored as log10 values

    # read the LSR spectrum: 
    wl_lsr = fh5['/PHOENIX_SPECTRUM_LSR/wl'][()]        # wavelength in Angstroem, vacuum
    fl_lsr = 10.**fh5['/PHOENIX_SPECTRUM_LSR/fl'][()]   # flux, stored as log10 values

    # Read the namelist
    target_nml = f90nml.reads(str(fh5['/PHOENIX_NAMELIST/phoenix_nml'][()].tobytes())[2:-1])

    # this is the stdout file (as a string):
    stdout_str = (str(fh5['/PHOENIX_STDOUT/phx_stdout'][()].tobytes()))[2:-1]

    # this is the restart file (as a string):
    restart_str = (str(fh5['/PHOENIX_RESTART/phx_restart'][()].tobytes()))[2:-1]

In [6]:
# Print out some contents from the namelist
for key in ["teff", "logg", "m_sun", "mixlng", "zscale", "alpha_scale"]:
    print(f"{key} = {target_nml['phoenix'][key]}")

teff = 12000.0
logg = 3.5
m_sun = 5.19029582884
mixlng = 1.0
zscale = 0.0
alpha_scale = 0.0


In [7]:
print((wl * u.Angstrom).to(u.um))
print(len(wl))

[9.00000000e-02 9.00000625e-02 9.00001250e-02 ... 7.30000000e+02
 8.30000000e+02 9.30000000e+02] um
13149991


In [8]:
print((wl_lsr * u.Angstrom).to(u.um))
print(len(wl_lsr))

[1.00e-03 1.01e-03 1.02e-03 ... 8.50e+02 9.00e+02 9.50e+02] um
473418


In [9]:
# now extract some values:
teff = target_nml['phoenix']['teff']
r0 = target_nml['phoenix']['r0']
v0 = target_nml['phoenix']['v0']
logg = target_nml['phoenix']['logg']
zscale = target_nml['phoenix']['zscale']
alpha_scale = target_nml['phoenix']['alpha_scale']
m_sun  = target_nml['phoenix']['m_sun']
wltau  = target_nml['phoenix']['wltau']
ngrrad  = target_nml['phoenix']['ngrrad']
ieos  = target_nml['phoenix']['ieos']
mixlng  = target_nml['phoenix']['mixlng']